In [1]:
!pip install osmnx h3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.4 MB/s eta 0:00:00


In [2]:
import h3
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely import wkt
from functools import reduce
from datetime import datetime
from typing import Optional

In [3]:
def parse_attended_people(attended: str) -> int:
    try:
        return int(attended)
    except:
        return np.nan

def parse_integer(value: str) -> Optional[int]:
    try:
        return int(value)
    except:
        return np.nan

# Procesamiento datos B1

In [4]:
b1_01 = pd.read_csv(
  filepath_or_buffer="./data/raw/ConsultaB1_ProduccionAsistencial_2023_01_v2.csv",
  sep=";",
  encoding="latin-1",
  dtype={"CO_IPRESS": str}
)

b1_02 = pd.read_csv(
  filepath_or_buffer="./data/raw/ConsultaB1_ProduccionAsistencial_2023_02_v2.csv",
  sep=";",
  encoding="latin-1",
  dtype={"CO_IPRESS": str}
)

b1_03 = pd.read_csv(
  filepath_or_buffer="./data/raw/ConsultaB1_ProduccionAsistencial_2023_03_v2.csv",
  sep=";",
  encoding="latin-1",
  dtype={"CO_IPRESS": str}
)

b1_04 = pd.read_csv(
  filepath_or_buffer="./data/raw/ConsultaB1_ProduccionAsistencial_2023_04_v1.csv",
  sep=";",
  encoding="latin-1",
  dtype={"CO_IPRESS": str}
)

In [5]:
b1_01["PERSONAS_ATENDIDAS"] = b1_01["PERSONAS_ATENDIDAS"].apply(parse_attended_people)
b1_01 = b1_01[~(b1_01["PERSONAS_ATENDIDAS"].isnull())]
b1_01 = b1_01.groupby("CO_IPRESS", as_index=False).agg(
  attended_people=("PERSONAS_ATENDIDAS", sum)
)

b1_02["PERSONAS_ATENDIDAS"] = b1_02["PERSONAS_ATENDIDAS"].apply(parse_attended_people)
b1_02 = b1_02[~(b1_02["PERSONAS_ATENDIDAS"].isnull())]
b1_02 = b1_02.groupby("CO_IPRESS", as_index=False).agg(
  attended_people=("PERSONAS_ATENDIDAS", sum)
)

b1_03["PERSONAS_ATENDIDAS"] = b1_03["PERSONAS_ATENDIDAS"].apply(parse_attended_people)
b1_03 = b1_03[~(b1_03["PERSONAS_ATENDIDAS"].isnull())]
b1_03 = b1_03.groupby("CO_IPRESS", as_index=False).agg(
  attended_people=("PERSONAS_ATENDIDAS", sum)
)

b1_04["PERSONAS_ATENDIDAS"] = b1_04["PERSONAS_ATENDIDAS"].apply(parse_attended_people)
b1_04 = b1_04[~(b1_04["PERSONAS_ATENDIDAS"].isnull())]
b1_04 = b1_04.groupby("CO_IPRESS", as_index=False).agg(
  attended_people=("PERSONAS_ATENDIDAS", sum)
)

In [6]:
b1_final = pd.concat([b1_01, b1_02, b1_03, b1_04])
b1_final = b1_final.groupby("CO_IPRESS", as_index=False).agg(
  attended_people=("attended_people", sum)
)
b1_final

,CO_IPRESS,attended_people
0,00000001,5573.0
1,00000003,9151.0
2,00000066,5152.0
3,00000162,3561.0
4,00000210,5833.0
...,...,...
2857,00031567,1250.0
2858,00031617,3248.0
2859,00031804,1011.0
2860,00032056,3945.0


# Procesamiento datos B2

In [7]:
b2_01 = pd.read_csv(
  filepath_or_buffer="./data/raw/ConsultaB2_Morbilidad_2023_01_v2.csv",
  sep=";",
  encoding="latin-1",
  dtype={"CO_IPRESS": str},
  low_memory=False
)

b2_02 = pd.read_csv(
  filepath_or_buffer="./data/raw/ConsultaB2_Morbilidad_2023_02_v2.csv",
  sep=";",
  encoding="latin-1",
  dtype={"CO_IPRESS": str},
  low_memory=False
)

b2_03 = pd.read_csv(
  filepath_or_buffer="./data/raw/ConsultaB2_Morbilidad_2023_03_v2.csv",
  sep=";",
  encoding="latin-1",
  dtype={"CO_IPRESS": str},
  low_memory=False
)

b2_04 = pd.read_csv(
  filepath_or_buffer="./data/raw/ConsultaB2_Morbilidad_2023_04_v1.csv",
  sep=";",
  encoding="latin-1",
  dtype={"CO_IPRESS": str},
  low_memory=False
)

In [8]:
b2_01["NU_TOTAL_ATENDIDOS"] = b2_01["NU_TOTAL_ATENDIDOS"].apply(parse_attended_people)
b2_01 = b2_01[~(b2_01["NU_TOTAL_ATENDIDOS"].isnull())]
b2_01 = b2_01.groupby("CO_IPRESS", as_index=False).agg(
  attended_people=("NU_TOTAL_ATENDIDOS", sum)
)

b2_02["NU_TOTAL_ATENDIDOS"] = b2_02["NU_TOTAL_ATENDIDOS"].apply(parse_attended_people)
b2_02 = b2_02[~(b2_02["NU_TOTAL_ATENDIDOS"].isnull())]
b2_02 = b2_02.groupby("CO_IPRESS", as_index=False).agg(
  attended_people=("NU_TOTAL_ATENDIDOS", sum)
)

b2_03["NU_TOTAL_ATENDIDOS"] = b2_03["NU_TOTAL_ATENDIDOS"].apply(parse_attended_people)
b2_03 = b2_03[~(b2_03["NU_TOTAL_ATENDIDOS"].isnull())]
b2_03 = b2_03.groupby("CO_IPRESS", as_index=False).agg(
  attended_people=("NU_TOTAL_ATENDIDOS", sum)
)

b2_04["NU_TOTAL_ATENDIDOS"] = b2_04["NU_TOTAL_ATENDIDOS"].apply(parse_attended_people)
b2_04 = b2_04[~(b2_04["NU_TOTAL_ATENDIDOS"].isnull())]
b2_04 = b2_04.groupby("CO_IPRESS", as_index=False).agg(
  attended_people=("NU_TOTAL_ATENDIDOS", sum)
)

In [9]:
b2_final = pd.concat([b2_01, b2_02, b2_03, b2_04])
b2_final = b2_final.groupby("CO_IPRESS", as_index=False).agg(
  attended_people=("attended_people", sum)
)
b2_final

,CO_IPRESS,attended_people
0,00000001,1900.0
1,00000003,5652.0
2,00000066,4815.0
3,00000162,35973.0
4,00000210,356.0
...,...,...
2800,00031567,1501.0
2801,00031617,5505.0
2802,00031804,895.0
2803,00032056,2908.0


# Procesamiento de datos C2

In [10]:
c2_01 = pd.read_csv(
  filepath_or_buffer="./data/raw/ConsultaC2_Morbilidad_2023_01_v2.csv",
  sep=";",
  encoding="latin-1",
  dtype={"CO_IPRESS": str}
)

c2_02 = pd.read_csv(
  filepath_or_buffer="./data/raw/ConsultaC2_Morbilidad_2023_02_v4.csv",
  sep=";",
  encoding="latin-1",
  dtype={"CO_IPRESS": str}
)

c2_04 = pd.read_csv(
  filepath_or_buffer="./data/raw/ConsultaC2_Morbilidad_2023_04_v1_0.csv",
  sep=";",
  encoding="latin-1",
  dtype={"CO_IPRESS": str}
)

In [11]:
c2_01["NU_TOTAL_ATENDIDOS"] = c2_01["NU_TOTAL_ATENDIDOS"].apply(parse_attended_people)
c2_01 = c2_01[~(c2_01["NU_TOTAL_ATENDIDOS"].isnull())]
c2_01 = c2_01.groupby("CO_IPRESS", as_index=False).agg(
  attended_people=("NU_TOTAL_ATENDIDOS", sum)
)

c2_02["NU_TOTAL_ATENDIDOS"] = c2_02["NU_TOTAL_ATENDIDOS"].apply(parse_attended_people)
c2_02 = c2_02[~(c2_02["NU_TOTAL_ATENDIDOS"].isnull())]
c2_02 = c2_02.groupby("CO_IPRESS", as_index=False).agg(
  attended_people=("NU_TOTAL_ATENDIDOS", sum)
)


c2_04["NU_TOTAL_ATENDIDOS"] = c2_04["NU_TOTAL_ATENDIDOS"].apply(parse_attended_people)
c2_04 = c2_04[~(c2_04["NU_TOTAL_ATENDIDOS"].isnull())]
c2_04 = c2_04.groupby("CO_IPRESS", as_index=False).agg(
  attended_people=("NU_TOTAL_ATENDIDOS", sum)
)

In [12]:
c2_final = pd.concat([c2_01, c2_02, c2_04])
c2_final = c2_final.groupby("CO_IPRESS", as_index=False).agg(
  attended_people=("attended_people", sum)
)
c2_final

,CO_IPRESS,attended_people
0,00000001,14569.0
1,00000003,3954.0
2,00000066,159.0
3,00000162,436.0
4,00000210,2676.0
...,...,...
745,00028448,83.0
746,00028966,15.0
747,00030057,3517.0
748,00030839,18.0


# Total de personas atendidas

In [13]:
attended = pd.concat([b1_final, b2_final, c2_final])
attended = attended.groupby("CO_IPRESS", as_index=False).agg(
    attended_people=("attended_people", sum)
)
attended

,CO_IPRESS,attended_people
0,00000001,22042.0
1,00000003,18757.0
2,00000066,10126.0
3,00000162,39970.0
4,00000210,8865.0
...,...,...
2865,00031567,2751.0
2866,00031617,8753.0
2867,00031804,1906.0
2868,00032056,6853.0


In [14]:
attended.to_csv("./data/outputs/attended_people.csv", index=False)

# Procesmiento información IPRESS

In [15]:
ipress = pd.read_csv(
  filepath_or_buffer="./data/raw/RENIPRESS_2023_v9.csv",
  sep=";",
  encoding="latin-1",
  dtype={"COD_IPRESS": str},
  low_memory=False
)

In [16]:
ipress = ipress[ipress["CATEGORIA"].isin(["II-1", "II-2", "III-1", "III-2", "III-E"])]
ipress = ipress[ipress["PROVINCIA"].isin(["LIMA", "CALLAO"])]
ipress = ipress[~(ipress["NORTE"].isnull())]
ipress.rename(columns={"NORTE": "lat", "ESTE": "lon", "COD_IPRESS": "CO_IPRESS"}, inplace=True)
ipress

,INSTITUCION,CO_IPRESS,NOMBRE,CLASIFICACION,TIPO_ESTABLECIMIENTO,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,DIRECCION,...,SITUACION,CONDICION,lat,lon,IMAGEN_1,FE_ACT_IMAGEN_1,IMAGEN_2,FE_ACT_IMAGEN_2,IMAGEN_3,FE_ACT_IMAGEN_3
4112,MINSA,00005946,HOSPITAL NACIONAL HIPOLITO UNANUE,HOSPITALES O CLINICAS DE ATENCION GENERAL,ESTABLECIMIENTO DE SALUD CON INTERNAMIENTO,LIMA,LIMA,EL AGUSTINO,150111,AV. CÉSAR VALLEJO 1390 - EL AGUSTINO,...,REGISTRADO,ACTIVO,-12.039969,-76.992582,http://app20.susalud.gob.pe:8080/registro-reni...,NaN,http://app20.susalud.gob.pe:8080/registro-reni...,NaN,http://app20.susalud.gob.pe:8080/registro-reni...,NaN
4113,MINSA,00005947,HOSPITAL DE MEDIANA COMPLEJIDAD JOSE AGURTO TELLO,HOSPITALES O CLINICAS DE ATENCION GENERAL,ESTABLECIMIENTO DE SALUD CON INTERNAMIENTO,LIMA,LIMA,LURIGANCHO,150118,JR. AREQUIPA 214-218,...,REGISTRADO,ACTIVO,-11.934195,-76.693495,http://app20.susalud.gob.pe:8080/registro-reni...,NaN,http://app20.susalud.gob.pe:8080/registro-reni...,NaN,http://app20.susalud.gob.pe:8080/registro-reni...,NaN
4221,MINSA,00005617,HOSPITAL SAN JUAN DE LURIGANCHO,HOSPITALES O CLINICAS DE ATENCION GENERAL,ESTABLECIMIENTO DE SALUD CON INTERNAMIENTO,LIMA,LIMA,SAN JUAN DE LURIGANCHO,150132,ENTRE PARADERO 10 Y 11 DE AV. CANTO GRANDE (HU...,...,REGISTRADO,ACTIVO,-11.966577,-77.003057,http://app20.susalud.gob.pe:8080/registro-reni...,NaN,http://app20.susalud.gob.pe:8080/registro-reni...,NaN,http://app20.susalud.gob.pe:8080/registro-reni...,NaN
4323,MINSA,00005987,HOSPITAL MARIA AUXILIADORA,HOSPITALES O CLINICAS DE ATENCION GENERAL,ESTABLECIMIENTO DE SALUD CON INTERNAMIENTO,LIMA,LIMA,SAN JUAN DE MIRAFLORES,150133,CALLE MIGUEL IGLESIAS N°968,...,REGISTRADO,ACTIVO,-12.160577,-76.959102,http://app20.susalud.gob.pe:8080/registro-reni...,NaN,http://app20.susalud.gob.pe:8080/registro-reni...,NaN,http://app20.susalud.gob.pe:8080/registro-reni...,NaN
4442,MINSA,00005883,HOSPITAL DE HUAYCAN,HOSPITALES O CLINICAS DE ATENCION GENERAL,ESTABLECIMIENTO DE SALUD CON INTERNAMIENTO,LIMA,LIMA,ATE,150103,JOSÉ C.MARIÁTEGUI S/N ZONA B - A.H.HUAYCÁN,...,REGISTRADO,ACTIVO,-12.015889,-76.820118,http://app20.susalud.gob.pe:8080/registro-reni...,NaN,http://app20.susalud.gob.pe:8080/registro-reni...,NaN,http://app20.susalud.gob.pe:8080/registro-reni...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21149,MINSA,00005948,HOSPITAL HERMILIO VALDIZAN,HOSPITALES O CLINICAS DE ATENCION GENERAL,ESTABLECIMIENTO DE SALUD CON INTERNAMIENTO,LIMA,LIMA,SANTA ANITA,150137,CARRETERA CENTRAL KM 3.5 SANTA ANITA,...,REGISTRADO,ACTIVO,-12.046827,-76.945498,http://app20.susalud.gob.pe:8080/registro-reni...,NaN,NaN,NaN,NaN,NaN
21156,ESSALUD,00010135,HOSPITAL I CARLOS ALCANTARA BUTTERFIELD,HOSPITALES O CLINICAS DE ATENCION GENERAL,ESTABLECIMIENTO DE SALUD CON INTERNAMIENTO,LIMA,LIMA,LA MOLINA,150114,AV. LOS CONSTRUCTORES 1201 - URB. COVIMA,...,REGISTRADO,ACTIVO,-12.062131,-76.945544,http://app20.susalud.gob.pe:8080/registro-reni...,NaN,http://app20.susalud.gob.pe:8080/registro-reni...,NaN,http://app20.susalud.gob.pe:8080/registro-reni...,NaN
21435,PRIVADO,00009123,CLINICA ANGLOAMERICANA,HOSPITALES O CLINICAS DE ATENCION GENERAL,ESTABLECIMIENTO DE SALUD CON INTERNAMIENTO,LIMA,LIMA,SAN ISIDRO,150131,CALLE ALFREDO SALAZAR Nº 350,...,REGISTRADO,ACTIVO,-12.108503,-77.038518,http://app20.susalud.gob.pe:8080/registro-reni...,NaN,http://app20.susalud.gob.pe:8080/registro-reni...,NaN,http://app20.susalud.gob.pe:8080/registro-reni...,NaN
21855,PRIVADO,00016291,HOSPITAL MEGASALUD UNIVERSAL,HOSPITALES O CLINICAS DE ATENCION ESPECIALIZADA,ESTABLECIMIENTO DE SALUD CON INTERNAMIENTO,LIMA,LIMA,LIMA,150101,AVENIDA EMANCIPACION 785-791-799,...,REGISTRADO,ACTIVO,-12.044736,-77.040195,http://app20.susalud.gob.pe:8080/registro-reni...,NaN,NaN,NaN,NaN,NaN


# Procesamiento recursos IPRESS

In [17]:
resources = pd.read_csv(
  filepath_or_buffer="./data/raw/ConsultaA_Recursos_Salud_2023_v6.csv",
  sep=";",
  encoding="latin-1",
  dtype={"CO_IPRESS": str},
  low_memory=False
)

In [18]:
resources_columns = [
  "CO_IPRESS", "CA_CONSULTORIOS", "CA_CONSULTORIOS_FN", "CA_CAMAS", "CA_MEDICOS_TOTAL", "CA_MEDICOS_SERUM",
  "CA_MEDICOS_RESIDENTES", "CA_ENFERMERAS", "CA_ODONTOLOGOS", "CA_PSICOLOGOS", "CA_NUTRICIONISTAS",
  "CA_TECNOLOGOS_MEDICOS", "CA_OBSTETRICES", "CA_FARMACEUTICOS", "CA_AUXILIARES", "CA_OTROS_PROF",
  "CA_AMBULANCIAS"
]
resources.sort_values(by=["CO_IPRESS", "ANHO", "MES"], inplace=True)
resources = resources[resources_columns].copy()
for c in resources.columns:
  if c != "CO_IPRESS":
    resources[c] = resources[c].apply(parse_integer)
resources.drop_duplicates(subset=["CO_IPRESS"], keep="last", inplace=True)
resources

,CO_IPRESS,CA_CONSULTORIOS,CA_CONSULTORIOS_FN,CA_CAMAS,CA_MEDICOS_TOTAL,CA_MEDICOS_SERUM,CA_MEDICOS_RESIDENTES,CA_ENFERMERAS,CA_ODONTOLOGOS,CA_PSICOLOGOS,CA_NUTRICIONISTAS,CA_TECNOLOGOS_MEDICOS,CA_OBSTETRICES,CA_FARMACEUTICOS,CA_AUXILIARES,CA_OTROS_PROF,CA_AMBULANCIAS
2565,00000001,31.0,33.0,95.0,68.0,0.0,2.0,140.0,4.0,5.0,6.0,6.0,54.0,15.0,277.0,8.0,2.0
8319,00000003,33.0,34.0,214.0,101.0,0.0,12.0,270.0,8.0,7.0,4.0,16.0,54.0,30.0,688.0,40.0,1.0
1629,00000066,15.0,15.0,32.0,2.0,0.0,0.0,8.0,3.0,2.0,0.0,2.0,4.0,0.0,32.0,2.0,1.0
5122,00000162,4.0,4.0,30.0,5.0,1.0,2.0,9.0,1.0,1.0,0.0,0.0,20.0,5.0,25.0,5.0,5.0
986,00000210,13.0,0.0,67.0,48.0,3.0,0.0,78.0,2.0,3.0,2.0,4.0,30.0,8.0,79.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4304,00031804,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1587,00031894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4874,00032056,10.0,20.0,0.0,5.0,1.0,0.0,10.0,3.0,3.0,2.0,1.0,5.0,0.0,14.0,3.0,1.0
7005,00032181,5.0,5.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


# Construcción destinaciones sin tomar en cuenta grafo

In [19]:
density = gpd.read_file("./data/external/density.gpkg")
density["hex_id"] = density["h3"]
density = density[["hex_id", "population"]]
density.rename(columns={"population": "population_dest"}, inplace=True)

In [20]:
destinations = pd.merge(ipress, resources, how="left", on="CO_IPRESS")
destinations = destinations[~(destinations["CA_CONSULTORIOS"].isnull())]
destinations = pd.merge(destinations, attended, how="left", on="CO_IPRESS")
destinations = destinations[~(destinations["attended_people"].isnull())]
destinations["INICIO_ACTIVIDAD"] = pd.to_datetime(destinations["INICIO_ACTIVIDAD"], format="%d/%m/%Y")
destinations["age"] = (datetime.now() - destinations["INICIO_ACTIVIDAD"]) // pd.Timedelta(days=365.25)
destinations = destinations[destinations["CA_CONSULTORIOS_FN"] > 0]
destinations["hex_id"] = destinations.apply(
    lambda x: h3.geo_to_h3(x.get("lat"), x.get("lon"), 8),
    axis=1
)
destinations = pd.merge(destinations, density, how="left", on="hex_id")
destinations

,INSTITUCION,CO_IPRESS,NOMBRE,CLASIFICACION,TIPO_ESTABLECIMIENTO,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,DIRECCION,...,CA_TECNOLOGOS_MEDICOS,CA_OBSTETRICES,CA_FARMACEUTICOS,CA_AUXILIARES,CA_OTROS_PROF,CA_AMBULANCIAS,attended_people,age,hex_id,population_dest
0,MINSA,00005946,HOSPITAL NACIONAL HIPOLITO UNANUE,HOSPITALES O CLINICAS DE ATENCION GENERAL,ESTABLECIMIENTO DE SALUD CON INTERNAMIENTO,LIMA,LIMA,EL AGUSTINO,150111,AV. CÉSAR VALLEJO 1390 - EL AGUSTINO,...,127.0,56.0,52.0,1151.0,406.0,4.0,88114.0,74,888e62c291fffff,19757.0
1,MINSA,00005947,HOSPITAL DE MEDIANA COMPLEJIDAD JOSE AGURTO TELLO,HOSPITALES O CLINICAS DE ATENCION GENERAL,ESTABLECIMIENTO DE SALUD CON INTERNAMIENTO,LIMA,LIMA,LURIGANCHO,150118,JR. AREQUIPA 214-218,...,20.0,26.0,16.0,216.0,7.0,4.0,49003.0,36,888e62832dfffff,4657.0
2,MINSA,00005617,HOSPITAL SAN JUAN DE LURIGANCHO,HOSPITALES O CLINICAS DE ATENCION GENERAL,ESTABLECIMIENTO DE SALUD CON INTERNAMIENTO,LIMA,LIMA,SAN JUAN DE LURIGANCHO,150132,ENTRE PARADERO 10 Y 11 DE AV. CANTO GRANDE (HU...,...,40.0,68.0,26.0,411.0,536.0,4.0,188019.0,47,888e62d529fffff,15316.0
3,MINSA,00005987,HOSPITAL MARIA AUXILIADORA,HOSPITALES O CLINICAS DE ATENCION GENERAL,ESTABLECIMIENTO DE SALUD CON INTERNAMIENTO,LIMA,LIMA,SAN JUAN DE MIRAFLORES,150133,CALLE MIGUEL IGLESIAS N°968,...,78.0,50.0,31.0,643.0,23.0,3.0,184874.0,39,888e62c5c1fffff,17511.0
4,MINSA,00005883,HOSPITAL DE HUAYCAN,HOSPITALES O CLINICAS DE ATENCION GENERAL,ESTABLECIMIENTO DE SALUD CON INTERNAMIENTO,LIMA,LIMA,ATE,150103,JOSÉ C.MARIÁTEGUI S/N ZONA B - A.H.HUAYCÁN,...,8.0,37.0,9.0,92.0,9.0,1.0,101668.0,37,888e628b07fffff,13852.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,MINSA,00005948,HOSPITAL HERMILIO VALDIZAN,HOSPITALES O CLINICAS DE ATENCION GENERAL,ESTABLECIMIENTO DE SALUD CON INTERNAMIENTO,LIMA,LIMA,SANTA ANITA,150137,CARRETERA CENTRAL KM 3.5 SANTA ANITA,...,4.0,0.0,13.0,150.0,15.0,2.0,57156.0,61,888e62d5b3fffff,13610.0
72,ESSALUD,00010135,HOSPITAL I CARLOS ALCANTARA BUTTERFIELD,HOSPITALES O CLINICAS DE ATENCION GENERAL,ESTABLECIMIENTO DE SALUD CON INTERNAMIENTO,LIMA,LIMA,LA MOLINA,150114,AV. LOS CONSTRUCTORES 1201 - URB. COVIMA,...,35.0,20.0,11.0,124.0,3.0,0.0,157774.0,23,888e62c75bfffff,8891.0
73,PRIVADO,00009123,CLINICA ANGLOAMERICANA,HOSPITALES O CLINICAS DE ATENCION GENERAL,ESTABLECIMIENTO DE SALUD CON INTERNAMIENTO,LIMA,LIMA,SAN ISIDRO,150131,CALLE ALFREDO SALAZAR Nº 350,...,29.0,5.0,7.0,198.0,1.0,0.0,37985.0,102,888e62c0b1fffff,5026.0
74,PRIVADO,00016291,HOSPITAL MEGASALUD UNIVERSAL,HOSPITALES O CLINICAS DE ATENCION ESPECIALIZADA,ESTABLECIMIENTO DE SALUD CON INTERNAMIENTO,LIMA,LIMA,LIMA,150101,AVENIDA EMANCIPACION 785-791-799,...,2.0,0.0,0.0,24.0,2.0,0.0,2945.0,10,888e62c01bfffff,10276.0


In [21]:
destinations.to_csv("./data/outputs/raw_destinations_hospitals.csv", index=False)

# Acoplar destinaciones al grafo

In [22]:
districts = gpd.read_file("./data/external/DISTRITOS.shp")
districts = districts[districts["PROVINCIA"].isin(["LIMA", "CALLAO"])].explode(index_parts=True).reset_index(drop=True)

In [24]:
nodes = pd.read_csv("data/external/GrafoLima_nodes.csv", index_col=[0])
nodes["geometry"] = nodes["geometry"].apply(wkt.loads)
nodes = gpd.GeoDataFrame(nodes, crs="epsg:4326")
nodes

,y,x,ref,highway,street_count,geometry
osmid,,,,,,
258066779,-12.371360,-76.784678,45A,motorway_junction,3,POINT (-76.78468 -12.37136)
258066780,-12.376318,-76.779559,NaN,NaN,3,POINT (-76.77956 -12.37632)
258066781,-12.384553,-76.771046,NaN,NaN,3,POINT (-76.77105 -12.38455)
258066783,-12.385783,-76.769768,NaN,NaN,3,POINT (-76.76977 -12.38578)
258067237,-12.304254,-76.840727,NaN,NaN,3,POINT (-76.84073 -12.30425)
...,...,...,...,...,...,...
11074106301,-11.822343,-77.161338,NaN,NaN,4,POINT (-77.16134 -11.82234)
11074106303,-11.821708,-77.161237,NaN,NaN,4,POINT (-77.16124 -11.82171)
11074106305,-11.821329,-77.161223,NaN,NaN,3,POINT (-77.16122 -11.82133)


In [25]:
edges = pd.read_csv("data/external/GrafoLima_edgeswaze.csv", index_col=[0,1,2])
edges["geometry"] = edges["geometry"].apply(wkt.loads)
edges = gpd.GeoDataFrame(edges, crs="epsg:4326")
edges

<ipython-input-25-a8c8cb5220b6>:1: DtypeWarning: Columns (5,7,13,14,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  edges = pd.read_csv("data/external/GrafoLima_edgeswaze.csv", index_col=[0,1,2])


osmid  oneway lanes  \
u           v           key                                                     
258066779   10985400003 0                              39457621    True     1   
            258066780   0    [1046533254, 338193454, 143630423]    True     2   
258066780   258066781   0                             143630430    True     2   
258066781   258066783   0                             143630430    True     2   
            495302399   0                              40752940    True     1   
...                                                         ...     ...   ...   
11074106305 11074106303 0                            1193021154   False   NaN   
11074106311 5497647609  0                             439736677   False   NaN   
11074106403 1273703683  0                             111938577   False   NaN   
            11001190815 0                             111931967   False   2.0   
            10738000177 0                             111938577   False   NaN   

                                   highway maxspeed    length  \
u           v           key                                     
258066779   10985400003 0    motorway_link       80   109.492   
            258066780   0         motorway      100   782.922   
258066780   258066781   0            trunk      100  1301.315   
258066781   258066783   0            trunk      100   194.871   
            495302399   0       trunk_link      NaN    85.154   
...                                    ...      ...       ...   
11074106305 11074106303 0      residential      NaN    42.177   
11074106311 5497647609  0      residential      NaN   126.496   
11074106403 1273703683  0      residential      NaN    51.814   
            11001190815 0      residential      NaN    61.585   
            10738000177 0      residential      NaN   113.466   

                                                                      geometry  \
u           v           key                                                      
258066779   10985400003 0    LINESTRING (-76.78468 -12.37136, -76.78466 -12...   
            258066780   0    LINESTRING (-76.78468 -12.37136, -76.78433 -12...   
258066780   258066781   0    LINESTRING (-76.77956 -12.37632, -76.77670 -12...   
258066781   258066783   0    LINESTRING (-76.77105 -12.38455, -76.76977 -12...   
            495302399   0    LINESTRING (-76.77105 -12.38455, -76.77099 -12...   
...                                                                        ...   
11074106305 11074106303 0    LINESTRING (-77.16122 -11.82133, -77.16124 -11...   
11074106311 5497647609  0    LINESTRING (-77.15013 -11.82304, -77.15002 -11...   
11074106403 1273703683  0    LINESTRING (-77.16524 -11.77607, -77.16571 -11...   
            11001190815 0    LINESTRING (-77.16524 -11.77607, -77.16521 -11...   
            10738000177 0    LINESTRING (-77.16524 -11.77607, -77.16487 -11...   

                               ref                        name bridge  \
u           v           key                                             
258066779   10985400003 0      NaN                         NaN    NaN   
            258066780   0    PE-1S  Carretera Panamericana Sur    yes   
258066780   258066781   0    PE-1S  Carretera Panamericana Sur    NaN   
258066781   258066783   0    PE-1S  Carretera Panamericana Sur    NaN   
            495302399   0      NaN                         NaN    NaN   
...                            ...                         ...    ...   
11074106305 11074106303 0      NaN                         NaN    NaN   
11074106311 5497647609  0      NaN               Calle Samiria    NaN   
11074106403 1273703683  0      NaN                         NaN    NaN   
            11001190815 0      NaN                         NaN    NaN   
            10738000177 0      NaN                         NaN    NaN   

                            junction access tunnel  width service  \
u           v           key                                         
258

In [26]:
graph = ox.graph_from_gdfs(gdf_nodes=nodes, gdf_edges=edges)
nodes = nodes.reset_index()
nodes.rename(columns={"y": "lat", "x": "lon"}, inplace=True)

In [27]:
new_destinations = destinations.copy()
new_destinations["osmid"] = new_destinations.apply(
    lambda x: ox.distance.nearest_nodes(graph, x.get("lon"), x.get("lat"), return_dist=False),
    axis=1
)
new_destinations.drop(columns=["lat", "lon"], inplace=True)
new_destinations = pd.merge(new_destinations, nodes, how="left", on="osmid")
new_destinations["hex_id"] = new_destinations.apply(
    lambda x: h3.geo_to_h3(x.get("lat"), x.get("lon"), 8),
    axis=1
)
new_destinations = new_destinations[[
    "CO_IPRESS",
    "NOMBRE",
    "DEPARTAMENTO",
    "PROVINCIA",
    "DISTRITO",
    "CATEGORIA",
    "CA_CONSULTORIOS",
    "CA_CONSULTORIOS_FN",
    "CA_CAMAS",
    "CA_MEDICOS_TOTAL",
    "attended_people",
    "age",
    "hex_id",
    "population_dest",
    "osmid",
    "lon",
    "lat"
]]
new_destinations

,CO_IPRESS,NOMBRE,DEPARTAMENTO,PROVINCIA,DISTRITO,CATEGORIA,CA_CONSULTORIOS,CA_CONSULTORIOS_FN,CA_CAMAS,CA_MEDICOS_TOTAL,attended_people,age,hex_id,population_dest,osmid,lon,lat
0,00005946,HOSPITAL NACIONAL HIPOLITO UNANUE,LIMA,LIMA,EL AGUSTINO,III-1,91.0,110.0,592.0,513.0,88114.0,74,888e62c291fffff,19757.0,1273969287,-76.992431,-12.039887
1,00005947,HOSPITAL DE MEDIANA COMPLEJIDAD JOSE AGURTO TELLO,LIMA,LIMA,LURIGANCHO,II-2,15.0,32.0,67.0,139.0,49003.0,36,888e62832dfffff,4657.0,1273794013,-76.693176,-11.934347
2,00005617,HOSPITAL SAN JUAN DE LURIGANCHO,LIMA,LIMA,SAN JUAN DE LURIGANCHO,II-2,41.0,82.0,145.0,200.0,188019.0,47,888e62d529fffff,15316.0,4356245534,-77.002921,-11.966359
3,00005987,HOSPITAL MARIA AUXILIADORA,LIMA,LIMA,SAN JUAN DE MIRAFLORES,III-1,109.0,80.0,409.0,283.0,184874.0,39,888e62c5c1fffff,17511.0,4354250483,-76.959165,-12.160821
4,00005883,HOSPITAL DE HUAYCAN,LIMA,LIMA,ATE,II-1,43.0,46.0,64.0,79.0,101668.0,37,888e628b07fffff,13852.0,1570070526,-76.820163,-12.015918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,00005948,HOSPITAL HERMILIO VALDIZAN,LIMA,LIMA,SANTA ANITA,III-1,55.0,84.0,159.0,99.0,57156.0,61,888e62d5b3fffff,13610.0,1726146781,-76.944475,-12.046471
72,00010135,HOSPITAL I CARLOS ALCANTARA BUTTERFIELD,LIMA,LIMA,LA MOLINA,II-1,20.0,46.0,37.0,166.0,157774.0,23,888e62c75bfffff,8891.0,8301948853,-76.945694,-12.062277
73,00009123,CLINICA ANGLOAMERICANA,LIMA,LIMA,SAN ISIDRO,II-2,69.0,138.0,93.0,185.0,37985.0,102,888e62c0b1fffff,5026.0,134380224,-77.038941,-12.108260
74,00016291,HOSPITAL MEGASALUD UNIVERSAL,LIMA,LIMA,LIMA,II-1,24.0,15.0,22.0,33.0,2945.0,10,888e62c01bfffff,10276.0,7184729957,-77.040263,-12.044689


In [28]:
new_destinations.to_csv("./data/outputs/destinations_hospitals.csv", index=False)